# "Creating the first Streamlit project"

- comments: true


[Streamlit](https://www.streamlit.io/) is a perfect resource to deploy and visualise your own data project.

By default you can run your projects on a local machine using

> streamlit run <name_app.py>

If you want to share your project to the world you need to apply to [share.streamlit.io](https://www.share.streamlit.io/).

Today I will share with you my dog classification app that I have released on streamlit.
It can be found [here](https://share.streamlit.io/byte5169/dogs_breed_repo/main/stream_app.py).

The basic idea is that you upload an image of a dog, or paste an url of an image of a dog, and the trained model predicts and outputs the scores for the breed of the dog.

Let's start from importing everything we need for our dog classifier. Mainly it is fastai library that was used to train the model and also will be used to predict results, and streamlit itself.

In [ ]:
import streamlit as st

from fastai.vision.all import *
from fastai.vision.widgets import *

from PIL import Image
from io import BytesIO


Next we will create different variables that will be used across all web-app.

In [ ]:
# makes page centered, or you can use 'wide'
st.set_page_config(layout="centered")
path = Path()

# importing model loader
learn_inf = load_learner('models/dogs_breed_standford_V2_20_01.pkl', cpu=True)

# image that will be used as header logo for our app
header = Image.open('logo.jpg')

# setting header width equal to page width
st.image(header, use_column_width=True)

Next we will put in the description of our page. Inside method ```write``` you can use markdown, that is very handy!

![]({{site.baseurl}}/images/streamlit/streamlit_header.png)

In [ ]:
st.write("""
# A simple dog breed classifier!

""")

st.write("""
Model that predicts one of the 120 dog breeds. \n
Data used for model training was taken from Kaggle's [Standford Dogs dataset](
https://www.kaggle.com/jwyang91/dog-breed-classification-using-fastai/)
***
""")

Now we will create our upload widget and field to paste url.
I really didn't like how it looked one on another (upload widget and url field)
and decided to break the page in to two columns for that

![]({{site.baseurl}}/images/streamlit/streamlit_columns.png)

In [ ]:
# creating columns
col1, col2 = st.beta_columns(2)

# creating upload widget and setting types of files we'd like to see here
upload_img = col1.file_uploader(type=['png', 'jpg', 'jpeg'], label='Upload here to predict from image')

# creating url paste field
url_img = col2.text_input(label='or enter url here:')

# creating button under url field that will trigger the prediction
but = col2.button(label='Press to predict from url')

After we will crate a few functions that will do the conversion of images and the predictions.

Please read the comments inside functions for more details.

In [ ]:
# function to predict the dog breed
def predict_dog(dog_image):
    # take an image and convert it to tensor
    img_t = tensor(dog_image)
    # get predictions
    pred, pred_idx, probs = learn_inf.predict(img_t)
    # apply some basic formatting to results
    pred = pred.split('-')[1].capitalize()
    # converting probabilities to presents
    probs = str(round((probs[pred_idx].item() * 100), 2)) + ' %'
    return pred, probs

# function that outputs results to the web-page, we pass here image directly
def render_result(source):
    # storing returns from 'predict_dog' function
    the_dog = predict_dog(source)
    # create text areas to display results
    st.text(f'Prediction: {the_dog[0]}')
    st.text(f'Probability: {the_dog[1]}')
    # output the uploaded image/url image in shape 128x128 to user
    st.image(source.to_thumb(128, 128))

# function that converts url to actual image that we can work with
def fetch_url_image(url):
    # receive url
    response = requests.get(url)
    # open image
    img = Image.open(BytesIO(response.content))
    return img

Now this is the main block for upload widget.

We need to check if there is an actual something that was uploaded into upload widget, and if there is something we want to output results.

In [ ]:
if upload_img is not None:
    render_result(Image.open(upload_img))

And here is the main code block for the url block that fetches data,
predicts the breed and outputs the results. We wrap it into try/except block incase something goes wrong.

In [ ]:
try:
    if but:
        img_to_pred = fetch_url_image(url_img)
        predict_dog(img_to_pred)
        render_result(img_to_pred)
except:
    st.write("Unexpected error:", st.error())

Before upload app we need not to forget about requirements.txt file in the repo, it will tell streamlit what libraries are used.
```
streamlit==0.74.1
fastai>=2
```

### PS

I do understand that the code is not perfect and probably it can be done in much more elegant ways especially function blocks and logic behind them,
but that is where my current skill level of Python and ML currently is.

